In [ ]:
!pip install jaxtyping
!pip install git+https://github.com/Deep-Learning-Profiling-Tools/triton-viz@v1
!export LC_ALL="en_US.UTF-8"

In [12]:
# don't run these commands, they only install triton basically, but we already have a compatible version of triton installed
# !curl --output "triton-3.0.0-cp310-cp310-linux_x86_64.whl" "https://dl.cloudsmith.io/public/test-wha/triton-puzzles/raw/files/triton-3.0.0-cp310-cp310-linux_x86_64.whl"

In [16]:
!pip show triton

Name: triton
Version: 2.3.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /usr/lib64/python3.12/site-packages
Requires: filelock
Required-by: triton-viz


In [1]:
!module load rocm/gfx9

In [1]:
import torch
import triton
from torch import Tensor
import triton.language as tl
import jaxtyping
from jaxtyping import Float32, Int32

In [2]:
torch.cuda.is_available()

True

In [3]:
@triton.jit
def add_kernel(x_ptr,  # *Pointer* to first input vector.
               y_ptr,  # *Pointer* to second input vector.
               output_ptr,  # *Pointer* to output vector.
               n_elements,  # Size of the vector.
               BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process.
               # NOTE: `constexpr` so it can be used as a shape value.
               ):
    # There are multiple 'programs' processing different data. We identify which program
    # we are here:
    pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0.
    # This program will process inputs that are offset from the initial data.
    # For instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers:
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # Create a mask to guard memory operations against out-of-bounds accesses.
    mask = offsets < n_elements
    # Load x and y from DRAM, masking out any extra elements in case the input is not a
    # multiple of the block size.
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    output = x + y
    # Write x + y back to DRAM.
    tl.store(output_ptr + offsets, output, mask=mask)

In [4]:
def add(x: torch.Tensor, y: torch.Tensor):
    # We need to preallocate the output.
    output = torch.empty_like(x)
    assert x.is_cuda and y.is_cuda and output.is_cuda
    n_elements = output.numel()
    # The SPMD launch grid denotes the number of kernel instances that run in parallel.
    # It is analogous to CUDA launch grids. It can be either Tuple[int], or Callable(metaparameters) -> Tuple[int].
    # In this case, we use a 1D grid where the size is the number of blocks:
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    # NOTE:
    #  - Each torch.tensor object is implicitly converted into a pointer to its first element.
    #  - `triton.jit`'ed functions can be indexed with a launch grid to obtain a callable GPU kernel.
    #  - Don't forget to pass meta-parameters as keywords arguments.
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)
    # We return a handle to z but, since `torch.cuda.synchronize()` hasn't been called, the kernel is still
    # running asynchronously at this point.
    return output

In [5]:
torch.manual_seed(0)
size = 98432
x = torch.rand(size, device='cuda')
y = torch.rand(size, device='cuda')
output_torch = x + y
output_triton = add(x, y)
print(output_torch)
print(output_triton)
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

SystemError: <built-in function load_binary> returned NULL without setting an exception

#### stuff

In [7]:
import math

BLOCK = 512

# This is a GPU kernel in Triton.
# Different instances of this
# function may run in parallel.
@triton.jit
def add(X, Y, Z, N):
   # In Triton, each kernel instance
   # executes block operations on a
   # single thread: there is no construct
   # analogous to threadIdx
   pid = tl.program_id(0)
   # block of indices
   idx = pid * BLOCK + tl.arange(0, BLOCK)
   mask = idx < N
   # Triton uses pointer arithmetics  
   # rather than indexing operators
   x = tl.load(X + idx, mask=mask)
   y = tl.load(Y + idx, mask=mask)
   tl.store(Z + idx, x + y, mask=mask)

size = 100000
x = torch.rand(size, device='cuda')
y = torch.rand(size, device='cuda')
z = torch.empty_like(x, device='cuda')

grid = (math.ceil(size / BLOCK),)
# no thread-block
output_torch = x + y
add[grid](x, y, z, x.shape[0])

print(output_torch)
print(output_triton)
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

SystemError: <built-in function load_binary> returned NULL without setting an exception

#### stuff

In [13]:
import torch
import triton
from torch import Tensor
import triton.language as tl
import jaxtyping
from jaxtyping import Float32, Int32

# M rows, N cols
@triton.jit
def collapserows(Y, stride_ym, X, stride_xm, stride_xn, M, N):
    # row index
    m = tl.program_id(0)
    # col indices
    # this specific kernel only works for matrices that 
    # have less than BLOCK_SIZE columns
    BLOCK_SIZE: tl.constexpr = 1024
    n = tl.arange(0, BLOCK_SIZE)
    # the memory address of all the elements
    # that we want to load can be computed as follows
    X = (X + m * stride_xm) + n * stride_xn
    # load input data; pad out-of-bounds elements with 0 
    x = tl.load(X, mask=n < N, other=0)
    y = tl.sum(x)
    # write back to Y
    Y = Y + m * stride_ym
    tl.store(Y, y)

import torch
# Allocate input/output tensors
X = torch.normal(0, 1, size=(400, 400), device='cuda')
Y = torch.zeros(X.shape[0], device='cuda')
# SPMD launch grid
grid = (X.shape[0], )
# enqueue GPU kernel
collapserows[grid](Y, Y.stride(0),
              X, X.stride(0), X.stride(1),
              X.shape[0]    , X.shape[1])

Y_torch = torch.sum(X, dim=1)

print(Y.shape, Y[:5])
print(Y_torch.shape, Y_torch[:5])
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(Y_torch - Y))}')

torch.Size([400]) tensor([ 14.2556,  10.9699,   4.5875, -19.7804, -42.4840], device='cuda:0')
torch.Size([400]) tensor([ 14.2556,  10.9699,   4.5875, -19.7804, -42.4840], device='cuda:0')
The maximum difference between torch and triton is 7.62939453125e-06


In [15]:
X = torch.normal(0, 1, size=(8192, 8192), device='cuda')
Y = torch.zeros(X.shape[0], device='cuda')
grid = (X.shape[0], )
# SPMD launch grid

for i in range(100000):
    Y_torch = torch.sum(X, dim=1)

In [ ]:
X = torch.normal(0, 1, size=(8192, 8192), device='cuda')
Y = torch.zeros(X.shape[0], device='cuda')
grid = (X.shape[0], )
# SPMD launch grid

for i in range(100000):
    collapserows[grid](Y, Y.stride(0),
              X, X.stride(0), X.stride(1),
              X.shape[0]    , X.shape[1])

In [8]:
%env HIP_FORCE_DEV_KERNARG=0

env: HIP_FORCE_DEV_KERNARG=0


In [ ]:
import torch
import triton
from torch import Tensor
import triton.language as tl
import jaxtyping
from jaxtyping import Float32, Int32

@triton.jit
def softmax(Y, stride_ym, stride_yn, X, stride_xm, stride_xn, M, N):
    # row index
    m = tl.program_id(0)
    # col indices
    # this specific kernel only works for matrices that 
    # have less than BLOCK_SIZE columns
    BLOCK_SIZE: tl.constexpr = 1024
    n = tl.arange(0, BLOCK_SIZE)
    # the memory address of all the elements
    # that we want to load can be computed as follows

    """
    X = (X + m * stride_xm) + n * stride_xn
    # load input data; pad out-of-bounds elements with 0 
    neginf = -float('inf')
    x = tl.load(X, mask=n < N, other=neginf)
    # compute numerically-stable softmax
    z = x - tl.max(x, axis=0)
    #num = tl.exp(z)
    #denom = tl.sum(num, axis=0)
    #denom = tl.sum(x, axis=0)
    #y = tl.exp(x)
    # write back to Y
    """

    # calculate e^(first elememnt)
    value = tl.exp(tl.load(X + m * stride_xm + 0 * stride_xn))
    
    Y = Y + m * stride_ym + n * stride_yn
    tl.store(Y, value, mask=n < N)
    
    """
    Y = Y + m * stride_ym + n * stride_yn
    y = tl.sin(x)
    #print("hi")
    #tl.printf("%d", (y.shape[0]))
    tl.store(Y, y, mask=n < N)
    """
    

import torch
# Allocate input/output tensors
# torch.zeros(512, 2, device='cuda')
X = torch.zeros(512, 1, device='cuda')#torch.normal(0, 1, size=(512, 1), device='cuda')
Y = torch.empty_like(X, device='cuda')

print(X.dtype)

# SPMD launch grid
grid = (X.shape[0], )
# enqueue GPU kernel
softmax[grid](Y, Y.stride(0), Y.stride(1), 
              X, X.stride(0), X.stride(1),
              X.shape[0]    , X.shape[1])

print(Y)

In [1]:
%env MIOPEN_ENABLE_LOGGING=yes
%env MIOPEN_ENABLE_LOGGING_CMD=yes
%env MIOPEN_LOG_LEVEL=6

env: MIOPEN_ENABLE_LOGGING=yes
env: MIOPEN_ENABLE_LOGGING_CMD=yes
env: MIOPEN_LOG_LEVEL=6


In [2]:
!echo $MIOPEN_ENABLE_LOGGING

yes


In [10]:
import torch
import triton
from torch import Tensor
import triton.language as tl
import jaxtyping
from jaxtyping import Float32, Int32

@triton.jit
def exp(Y, stride_yn, X, stride_xn, N):
    # row index
    n = tl.program_id(0)
    # col indices
    # this specific kernel only works for matrices that 
    # have less than BLOCK_SIZE columns
    BLOCK_SIZE: tl.constexpr = 1024
    n_block = tl.arange(0, BLOCK_SIZE)
    # the memory address of all the elements
    # that we want to load can be computed as follows

    # calculate e^(first elememnt)
    z = tl.load(X + n * stride_xn + n_block, mask=n_block < N)
    
    value = 1 + z + z * z / 2 + z * z * z / 6 + z * z * z * z / 24 + z * z * z * z * z / 120 + z * z * z * z * z * z / (120 * 6) + z * z * z * z * z * z * z / (120 * 6 * 7)

    Y = Y + n * stride_yn + n_block
    tl.store(Y, value, mask=n_block < N)
    
    """
    Y = Y + m * stride_ym + n * stride_yn
    y = tl.sin(x)
    #print("hi")
    #tl.printf("%d", (y.shape[0]))
    tl.store(Y, y, mask=n < N)
    """
    

import torch
# Allocate input/output tensors
# torch.zeros(512, 2, device='cuda')
X = torch.normal(0, 1, size=(524288,), device='cuda')
Y = torch.empty_like(X, device='cuda')

print(X.dtype)

# SPMD launch grid
grid = (X.shape[0], )
# enqueue GPU kernel
exp[grid](Y, Y.stride(0), 
              X, X.stride(0),
              X.shape[0])

torch_out = torch.exp(X)

print(Y)
print(torch_out)

print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(torch_out - Y))}')


torch.float32
tensor([0.1197, 0.6572, 0.7279,  ..., 3.1513, 1.7558, 0.4742], device='cuda:0')
tensor([0.1264, 0.6572, 0.7279,  ..., 3.1514, 1.7558, 0.4742], device='cuda:0')
The maximum difference between torch and triton is 9.727210998535156


In [17]:
# @title Setup

import triton_viz
import inspect
from triton_viz.interpreter import record_builder

def test(puzzle, puzzle_spec, nelem={}, B={"B0": 32}, viz=True):
    B = dict(B)
    if "N1" in nelem and "B1" not in B:
        B["B1"] = 32
    if "N2" in nelem and "B2" not in B:
        B["B2"] = 32

    triton_viz.interpreter.record_builder.reset()
    torch.manual_seed(0)
    signature = inspect.signature(puzzle_spec)
    args = {}
    for n, p in signature.parameters.items():
        print(p)
        args[n + "_ptr"] = ([d.size for d in p.annotation.dims], p)
    args["z_ptr"] = ([d.size for d in signature.return_annotation.dims], None)

    tt_args = []
    for k, (v, t) in args.items():
        tt_args.append(torch.rand(*v) - 0.5)
        if t is not None and t.annotation.dtypes[0] == "int32":
            tt_args[-1] = torch.randint(-100000, 100000, v)
    grid = lambda meta: (triton.cdiv(nelem["N0"], meta["B0"]),
                         triton.cdiv(nelem.get("N1", 1), meta.get("B1", 1)),
                         triton.cdiv(nelem.get("N2", 1), meta.get("B2", 1)))

    #for k, v in args.items():
    #    print(k, v)
    triton_viz.trace(puzzle)[grid](*tt_args, **B, **nelem)
    z = tt_args[-1]
    tt_args = tt_args[:-1]
    z_ = puzzle_spec(*tt_args)
    match = torch.allclose(z, z_, rtol=1e-3, atol=1e-3)
    print("Results match:",  match)
    failures = False
    if viz:
        failures = triton_viz.launch()
    if not match or failures:
        print("Invalid Access:", failures)
        print("Yours:", z)
        print("Spec:", z_)
        print(torch.isclose(z, z_))
        return
    # PUPPIES!
    from IPython.display import HTML
    import random
    print("Correct!")
    pups = [
    "2m78jPG",
    "pn1e9TO",
    "MQCIwzT",
    "udLK6FS",
    "ZNem5o3",
    "DS2IZ6K",
    "aydRUz8",
    "MVUdQYK",
    "kLvno0p",
    "wScLiVz",
    "Z0TII8i",
    "F1SChho",
    "9hRi2jN",
    "lvzRF3W",
    "fqHxOGI",
    "1xeUYme",
    "6tVqKyM",
    "CCxZ6Wr",
    "lMW0OPQ",
    "wHVpHVG",
    "Wj2PGRl",
    "HlaTE8H",
    "k5jALH0",
    "3V37Hqr",
    "Eq2uMTA",
    "Vy9JShx",
    "g9I2ZmK",
    "Nu4RH7f",
    "sWp0Dqd",
    "bRKfspn",
    "qawCMl5",
    "2F6j2B4",
    "fiJxCVA",
    "pCAIlxD",
    "zJx2skh",
    "2Gdl1u7",
    "aJJAY4c",
    "ros6RLC",
    "DKLBJh7",
    "eyxH0Wc",
    "rJEkEw4"]
    return HTML("""
    <video alt="test" controls autoplay=1>
        <source src="https://openpuppies.com/mp4/%s.mp4"  type="video/mp4">
    </video>
    """%(random.sample(pups, 1)[0]))

ImportError: cannot import name 'InterpretedFunction' from 'triton.interpreter' (/usr/lib64/python3.12/site-packages/triton/interpreter/__init__.py)

In [8]:
!pip show triton

Name: triton
Version: 2.3.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /usr/lib64/python3.12/site-packages
Requires: filelock
Required-by: triton-viz


In [4]:
import triton
triton.__file__

'/usr/lib64/python3.12/site-packages/triton/__init__.py'

In [9]:
from triton import interpreter
from triton.interpreter.interpreter import InterpretedFunction

ImportError: cannot import name 'InterpretedFunction' from 'triton.interpreter.interpreter' (/usr/lib64/python3.12/site-packages/triton/interpreter/interpreter.py)

In [10]:
dir(triton.interpreter.interpreter)

['AutotuneGridSelector',
 'AutotuneRunner',
 'DebuggerFunction',
 'ExecutionContext',
 'GridSelector',
 'MemoryMap',
 'TritonLangProxy',
 'Tuple',
 'WrappedTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_primitive_to_tensor',
 'attach_triton',
 'debugger_constexpr',
 'detach_triton',
 'get_proxy_method',
 'itertools',
 'lcore',
 'program_ids_from_grid',
 'random',
 'tl',
 'tl_method_backup',
 'torch',
 'torch_wrapper',
 'triton_debug_autotune']

In [11]:
!pip show triton

Name: triton
Version: 2.3.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /usr/lib64/python3.12/site-packages
Requires: filelock
Required-by: triton-viz


In [6]:
import pkgutil

def list_modules(namespace):
    """List all modules and submodules in a given namespace."""
    for importer, modname, ispkg in pkgutil.iter_modules(namespace.__path__):
        print(modname)
        print(dir(modname))
        if ispkg:
            subpkg = namespace.__name__ + '.' + modname
            print(f"Submodules of {subpkg}:")
            list_modules(__import__(subpkg))

# Example usage:
list_modules(triton.runtime)

autotuner
['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'removeprefix', 'removesuffix', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']
cache
['_

In [4]:
dir(triton.runtime)

['Autotuner',
 'Config',
 'Heuristics',
 'JITFunction',
 'KernelInterface',
 'MockTensor',
 'OutOfResources',
 'TensorWrapper',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'autotune',
 'autotuner',
 'cache',
 'driver',
 'heuristics',
 'jit',
 'reinterpret',
 'version_key']

In [ ]:
@triton.jit
def softmax(Y, stride_ym, stride_yn, X, stride_xm, stride_xn, M, N):
    # row index
    m = tl.program_id(0)
    # col indices
    # this specific kernel only works for matrices that 
    # have less than BLOCK_SIZE columns
    BLOCK_SIZE: tl.constexpr = 1024
    n = tl.arange(0, BLOCK_SIZE)
    # the memory address of all the elements
    # that we want to load can be computed as follows
    X = X + m * stride_xm + n * stride_xn
    # load input data; pad out-of-bounds elements with 0 
    x = tl.load(X, mask=n < N, other=-float('inf'))
    # compute numerically-stable softmax
    z = x - tl.max(x, axis=0)
    num = tl.exp(z)
    denom = tl.sum(num, axis=0)
    y = num / denom
    # write back to Y
    Y = Y + m * stride_ym + n * stride_yn
    tl.store(Y, y, mask=n < N)

import torch
# Allocate input/output tensors
X = torch.normal(0, 1, size=(512, 512), device='cuda')
Y = torch.empty_like(X, device='cuda')
# SPMD launch grid
grid = (X.shape[0], )
# enqueue GPU kernel
softmax[grid](Y, Y.stride(0), Y.stride(1), 
              X, X.stride(0), X.stride(1),
              X.shape[0]    , X.shape[1])

In [5]:
torch.version.hip

'6.0.32831-'

In [6]:
import triton.runtime as tr
import triton.runtime.driver as trd
import triton.runtime.jit as trjit

In [14]:
trjit.get_device_capability(0)

(9, 0)

In [10]:
dir(trjit)

['Callable',
 'DependenciesFinder',
 'Generic',
 'Iterable',
 'JITFunction',
 'KernelInterface',
 'List',
 'MockTensor',
 'Optional',
 'T',
 'TMAInfos',
 'TRITON_PATH',
 'TRITON_VERSION',
 'TensorWrapper',
 'TypeVar',
 'Union',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_normalize_ty',
 'annotations',
 'ast',
 'cast',
 'defaultdict',
 'division',
 'dtype',
 'functools',
 'get_backend',
 'get_cuda_stream',
 'get_current_device',
 'get_device_capability',
 'hashlib',
 'inspect',
 'jit',
 'namedtuple',
 'os',
 'overload',
 'path_to_ptxas',
 'reinterpret',
 'set_current_device',
 'subprocess',
 'textwrap',
 'version_key']

In [9]:
!pip show triton

Name: triton
Version: 2.3.0
Summary: A language and compiler for custom Deep Learning operations
Home-page: https://github.com/openai/triton/
Author: Philippe Tillet
Author-email: phil@openai.com
License: 
Location: /usr/lib64/python3.12/site-packages
Requires: filelock
Required-by: triton-viz


In [1]:
!pip show triton_viz

Name: triton-viz
Version: 0.1
Summary: A visualization tool for Triton
Home-page: https://github.com/Deep-Learning-Profiling-Tools/triton-viz
Author: Deep Learning Profiling Tools Team
Author-email: kzhou6@gmu.edu
License: 
Location: /home/kge/.local/lib/python3.12/site-packages
Editable project location: /home/kge/triton_viz_install/triton-viz
Requires: chalk-diagrams, gradio, pre-commit, pyarrow, pytest, setuptools, triton
Required-by: 


In [1]:
!pip install matplotlib pandas -q
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/rocm6.0/ -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
